In [2]:
import xarray as xr
from matplotlib import pyplot as plt
import numpy as np
%load_ext autoreload
%autoreload 2
import pei.myfunctions as mf
from xhistogram.xarray import histogram
from dask.diagnostics import ProgressBar

In [3]:
# Load WBT data
ds = xr.open_mfdataset('../data/processed/WBTdaily/WBTdailyens*.nc',combine='nested',concat_dim='ensemble',chunks={'time':1000})

# Load area data
land_area = xr.open_dataset('../data/processed/wbt.land_area')
land_mask = np.isfinite(land_area)

# Replace NaN with 0 
land_area_adj = land_area.where(land_mask,0).rename({'__xarray_dataarray_variable__':'land_area'})

In [25]:
# Dictionary to store binned DataArrays
ds_dict = {}

regions = ['India','Southeast Asia','Northern Oceania','Central America','Northern South America','Southern South America',
          'Southern China','Southern Europe','Middle East','Central Africa']

In [26]:
for region in regions:
    # Isolate WBT and area data for region
    ds_region = mf.slice_region(ds,region)
    area_region = mf.slice_region(land_area_adj,region)

    # Specify histogram variable
    var = ds_region['WBT']

    # Bins to divide data at each point in time
    bins = np.arange(-55,40,0.1)

    # Specify area-weight variable
    area_weights = area_region['land_area']

    # Histogram in lat and lon dimensions
    dist = histogram(var,bins=[bins],dim=['lat','lon'],weights=area_weights)

    # Sum distributions within each year
    dist_annual = dist.groupby('time.year').sum('time')

    # Divide by area under curve to get PDF
    dist_annual = dist_annual/dist_annual.sum('WBT_bin')
    
    # Add to dictionary
    ds_dict[region] = dist_annual

In [27]:
ds_byregion = xr.Dataset(ds_dict)
#ds_byregion.to_netcdf('../data/processed/WBT_binned.nc')